# Accessing and saving data

In this notebook, I will try to gather some commands to read data from files, and save python objects or data into other files.

## To do list:

I should learn how to load various kinds of objects:

    1) saved python objects
    2) excell sheets

## Loading .csv files with basic Python

If one has a .csv file with two columns `domain` and `id_user` (the headers are on the first line) one can load into a list of string, each string representing a line, as follow:

In [6]:
FILE_NAME = 'Toy_data/toy_data.csv'

In [34]:
columns = {}

In [35]:
l = []
with open(FILE_NAME) as f:
    for row in f:
        l.append(row)

As one can see, one has then to process each line manually to split it into different entries.

In [36]:
print('First line is stored as {} with type {} '.format(repr(l[0]), type(l[0])))
print('Second line is {} and its type is {}'.format(repr(l[1]), type(l[1])))

First line is stored as '"a.com",32\n' with type <class 'str'> 
Second line is '"a.com",25\n' and its type is <class 'str'>


## Loading .csv files with csv library

In [4]:
import csv

This library has the advantage over the `.readline` method of directly splitting each line into a list of substrings (one for each column in a table).

In [37]:
l = []
with open(FILE_NAME) as f:
    reader = csv.reader(f)
    for row in reader:
        l.append(row)

In [38]:
print('First line is stored as {} with type {} '.format(repr(l[0]), type(l[0])))
print('Second line is {} and its type is {}'.format(repr(l[1]), type(l[1])))
print('There are in total {} lines in the file.'.format(len(l)))

First line is stored as ['domain', 'user_id'] with type <class 'list'> 
Second line is ['a.com', '32'] and its type is <class 'list'>
There are in total 6 lines in the file.


One can actually give option to the readers, as explained here: https://docs.python.org/3.7/library/csv.html#csv-fmt-params In particular, this can be used to read other types of table formats like .tsv files.

The following cell is equivalent to what was done above (using default parameters for the reader):

In [39]:
l = []
with open(FILE_NAME) as f:
    reader = csv.reader(f, delimiter=',', lineterminator='\n')
    for row in reader:
        l.append(row)

In [29]:
print('First line is stored as {} with type {} '.format(repr(l[0]), type(l[0])))
print('Second line is {} and its type is {}'.format(repr(l[1]), type(l[1])))
print('There are in total {} lines in the file.'.format(len(l)))

First line is stored as ['domain', 'user_id'] with type <class 'list'> 
Second line is ['a.com', '32'] and its type is <class 'list'>
There are in total 6 lines in the file.


If we want to create a dictionary where each item of the dictionary corresponds to a column of the .csv file, we proceed as follow:

In [11]:
d1 = {}
col_names = []
with open(FILE_NAME) as f:
    reader = csv.reader(f)
    for row in reader:
        if d1:
            for i, value in enumerate(row):
                d1[col_names[i]].append(value)
        else:
            col_names = row
            d1 = {col: [] for col in row}
        

In [12]:
print(d1)

{'domain': ['a.com', 'b.com', 'a.com', 'a.com', 'b.com'], 'user_id': ['32', '21', '25', '21', '21']}


If instead we want to create a list of ordered dictionaries, each dictionary representing one event/line, we can proceed as follow:

In [41]:
l_dict = []
with open(FILE_NAME) as f:
    reader = csv.DictReader(f)
    for row in reader:
        l_dict.append(row)

In [45]:
print(l_dict)

[OrderedDict([('domain', 'a.com'), ('user_id', '32')]), OrderedDict([('domain', 'b.com'), ('user_id', '21')]), OrderedDict([('domain', 'a.com'), ('user_id', '25')]), OrderedDict([('domain', 'a.com'), ('user_id', '21')]), OrderedDict([('domain', 'b.com'), ('user_id', '21')])]


Note that by default, the elements of the first row are used as the field names for the dictionary. So the above code is equivalent to:

In [52]:
l_dict = []
with open(FILE_NAME) as f:
    reader = csv.DictReader(f, fieldnames=['domain', 'user_id'])
    for i, row in enumerate(reader):
        if i>0:
            l_dict.append(row)

In [53]:
print(l_dict)

[OrderedDict([('domain', 'a.com'), ('user_id', '32')]), OrderedDict([('domain', 'b.com'), ('user_id', '21')]), OrderedDict([('domain', 'a.com'), ('user_id', '25')]), OrderedDict([('domain', 'a.com'), ('user_id', '21')]), OrderedDict([('domain', 'b.com'), ('user_id', '21')])]


**Note:** Actually, it seems good to encapsulate the reading of the file inside a function or a class as explained here: https://medium.com/district-data-labs/simple-csv-data-wrangling-with-python-3496aa5d0a5e

The page also provide good implementation of a class to read a csv table and avoid loading it entirely on the RAM.

## Saving and Loading Python objects with Pickle

I found the next piece of code here: https://wiki.python.org/moin/UsingPickle

Save a dictionary into a pickle file.

In [1]:
import pickle
favorite_color = { "lion": "yellow", "kitty": "red" }
pickle.dump( favorite_color, open( "save.p", "wb" ) )

In [3]:
favorite_color2 = pickle.load( open( "save.p", "rb" ) )

## Loading a csv table with numpy

If a csv table contains only numeric values, one can use numpy to load it directly into a numpy array. There are plenty of options, in particular, one can choose what to do with missing values and which column to select. This being said, I don't see many advantages over using the csv library.

**Note:** This is not memory efficient as the whole data is loaded on the RAM at once. It should hence be avoided when dealing with big files (as expained here: https://medium.com/district-data-labs/simple-csv-data-wrangling-with-python-3496aa5d0a5e)

In [8]:
import numpy as np
import csv

In [7]:
FILE_NAME2 = '/home/aritz/Documents/CS_Programming_Machine_Learning/Programming/Python/Useful_scripts/Toy_data/table_numbers.csv'

In [11]:
with open(FILE_NAME2) as f:
    reader = csv.reader(f)
    for i, line in enumerate(reader):
        if i == 0:
            headers = line

In [12]:
headers

['col1', 'col2', 'col3']

In [14]:
data = np.genfromtxt(fname=FILE_NAME2, delimiter=',', skip_header=1, missing_values='NA', filling_values=0,
                     dtype='float64', names=headers)

In [17]:
data

array([(112., 0.13, 3.), (117., 1.2 , 1.), ( 54., 0.77, 2.),
       ( 99., 0.5 , 2.), (  0., 1.4 , 1.)],
      dtype=[('col1', '<f8'), ('col2', '<f8'), ('col3', '<f8')])

**Next task**: Look how I loaded things in my sentiment analysis project, in NLP coursera, in Rossman fastai.